In [1]:
!pip install hazm
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# !wget https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip
# !wget https://github.com/sobhe/hazm/releases/download/v0.5/resources-stanford.zip
# !unzip resources-0.5.zip -d resources
# !unzip resources-stanford.zip -d resources

In [3]:
from datasets import load_dataset, load_metric
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import hazm
import string
from hazm import *
import copy
from transformers import pipeline


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
documents_df = pd.read_pickle("/content/drive/MyDrive/CNN/documents_embeding_df_full.pkl")
dataset = pd.read_pickle("/content/drive/MyDrive/CNN/context_query_question_embeding_df_full.pkl")

In [6]:
dataset = dataset.drop(['search_query_embed'], axis=1)
dataset = dataset.drop(['question_embed'], axis=1)
dataset = dataset.drop(['context_embed'], axis=1)

In [7]:
documents_df = documents_df.drop(['document_embeds'], axis=1)

In [8]:
create_query_from_question = True
number_of_documets_retrival = 1
number_of_contexts_retrival = 1

In [9]:
def create_search_query(text):
    normalizer = Normalizer()
    lemmatizer = Lemmatizer()
    tagger = POSTagger(model='resources/postagger.model')
    chunker = Chunker(model='resources/chunker.model')
    # normalize = normalizer.normalize(text)
    words = word_tokenize(text)
    pos_tags = tagger.tag(words)
    tree = chunker.parse(pos_tags)
    query_words = []
    for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):
        for word, pos in subtree.leaves():
            query_words.append(word)#(lemmatizer.lemmatize(word))
    query = ' '.join(query_words)
    return query

In [10]:
question_id =121 #19999

In [11]:
Question = dataset["question"][question_id]
Answer = dataset["answers"][question_id]
title = dataset["title"][question_id]

In [12]:
print("Question = " + Question)
print("title = "+title)

Question = فوتبال به سبک امروزی در چه سالی آغاز شد؟
title = فوتبال


In [13]:
Answer

{'text': ['اوایل سال ۱۸۰۰ میلادی'], 'answer_start': [199]}

In [14]:
Query = create_search_query(Question)

In [15]:
Query

'فوتبال سبک امروزی چه سالی'

======================================================

In [16]:
# Create a TfidfVectorizer object and fit it to the documents
vectorizer = TfidfVectorizer(input='content', analyzer='word', norm=None, smooth_idf=True)
tfidf_matrix = vectorizer.fit_transform((doc for doc in documents_df['context'].tolist()))

# Get the document IDs from the original list of tuples
doc_ids = [title for title in documents_df['title'].tolist()]

In [17]:
output_titles=[]
query_vector = vectorizer.transform([Query])
# Compute the cosine similarity between the query vector and the document vectors
cosine_similarities = np.dot(query_vector, tfidf_matrix.T).toarray()[0]
# Sort the documents by their cosine similarity scores
sorted_indices = np.argsort(cosine_similarities)[::-1]
for j in range(number_of_documets_retrival):
  document_index = sorted_indices[j]
  document_id = doc_ids[document_index]
  output_titles.append(document_id)

In [18]:
output_titles

['فوتبال']

In [19]:
def get_contexts_of_titles(titles,dataset):
  paragraphs =pd.DataFrame()
  for title in titles:
    df = dataset[dataset["title"] == title]
    if paragraphs.empty:
      paragraphs = copy.deepcopy(df)
    else:
      paragraphs = paragraphs.append(df)
  paragraphs = paragraphs.reset_index(drop=True)
  return paragraphs

In [20]:
selected_paragraphs = get_contexts_of_titles(output_titles,dataset)

In [21]:
def unique(list1):
    unique_list = []
    for x in list1:
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

In [22]:
unique_paraghraphs = unique(selected_paragraphs['context'].tolist())

In [23]:
# Create a TfidfVectorizer object and fit it to the documents
vectorizer = TfidfVectorizer(input='content', analyzer='word', norm=None, smooth_idf=True)
tfidf_matrix = vectorizer.fit_transform((doc for doc in unique_paraghraphs))

# Get the document IDs from the original list of tuples
doc_ids = [title for title in unique_paraghraphs]

In [24]:
output_paragraphs=[]
Question_vector = vectorizer.transform([Question])
# Compute the cosine similarity between the query vector and the document vectors
cosine_similarities = np.dot(Question_vector, tfidf_matrix.T).toarray()[0]
# Sort the documents by their cosine similarity scores
sorted_indices = np.argsort(cosine_similarities)[::-1]
for j in range(number_of_contexts_retrival):
  document_index = sorted_indices[j]
  document_id = doc_ids[document_index]
  output_paragraphs.append(document_id)

In [25]:
output_paragraphs

['پیشینهٔ معاصر این بازی جهانی به بیش از ۱۰۰ سال پیش بازمی\u200cگردد؛ زمانی که این ورزش در انگلستان شکل\u200cگرفت و اتحادیه فوتبال انگلستان به وجود آمد و فوتبال به ورزشی رسمی تبدیل\u200cشد. فوتبال به سبک امروزی، در اوایل سال ۱۸۰۰ میلادی در هفت مدرسهٔ عمومی انگلستان آغاز گردید. شش مدرسه تقریباً همان فوتبال به سبک قدیمی را اجرا می\u200cکردند؛ اما مدرسهٔ راگبی انگلستان که در سال ۱۵۶۷ تأسیس\u200cشده بود، نسخه\u200cای متفاوت از فوتبال را ارائه\u200cداد. مدارس دیگر نیز قوانینی را ساخته و در پایان، بازی فوتبال مدرن را بنیان\u200cگذاری کردند. در این سال اندازه و وزن توپ\u200cها نیز استاندارد شد. در نخستین بازی ۹۰ دقیقه\u200cای فوتبال، دو تیم لندن و شفیلد در سال ۱۸۶۶ به رقابت با یک\u200cدیگر پرداختند.']

In [26]:
mrr_model = pipeline(task = "question-answering" ,model="/content/drive/MyDrive/CN/distilbert-fa-zwnj-base-on-pquad-dataset")
# mrr_model = pipeline(task = "question-answering" ,model="/content/drive/MyDrive/CN/xlm-roberta-base-on-pquad-dataset")

In [27]:
outputs = []
for paragraph in output_paragraphs:
  outputs.append(mrr_model(question=Question, context=paragraph))

In [28]:
outputs

[{'score': 0.5924777984619141,
  'start': 199,
  'end': 220,
  'answer': 'اوایل سال ۱۸۰۰ میلادی'}]